In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
import numpy
import pathlib


In [7]:
# путь для скачивания датасета
file_url = 'https://github.com/PacktWorkshops/The-Deep-Learning-Workshop/raw/master/Chapter03/Datasets/Activity3.02/fruits360.zip'

In [11]:
# скачиваем и распаковываем(extract) файл
zip_dir = tf.keras.utils.get_file('fruits360.zip', origin=file_url, extract=True)

82223104/82220233 [==============================] - 122s 1us/step


In [14]:
# пусть для хранения
path = pathlib.Path(zip_dir).parent / 'fruits360_filtered'

In [15]:
train_dir = path / 'Training'  # папка для train set
validation_dir = path / 'Test'  # папка для validation set

In [16]:
# кол-во тренировочных и валидационных данных
total_train = 11398
total_val = 4752

In [17]:
# создаем data generators с аугментацией данных
train_img_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')
val_img_gen = ImageDataGenerator(rescale=1./255)

In [24]:
# data generator for training set
train_data_gen = train_img_gen.flow_from_directory(batch_size=16, directory=train_dir,target_size=(100, 100))

Found 11398 images belonging to 120 classes.


In [25]:
# data generator for validational set
val_data_gen = val_img_gen.flow_from_directory(batch_size=16, directory=validation_dir, target_size=(100,100))

Found 4752 images belonging to 120 classes.


In [37]:
val_data_gen

In [32]:
features_train_len = 11398
features_test_len = 4752


In [18]:
# загружаем предобученную модель VGG16
from tensorflow.keras.applications import VGG16

img_dim = (100, 100, 3)  # размеры входных картинок
base_model = VGG16(input_shape=img_dim, weights='imagenet', include_top=False)  # настройки модели: incude_top=False, так как заменяем последний полносвязный слой другим, для распознавания своего датасета
base_model.trainable = False  # Замораживаем модель, чтобы ее можно было обучать в таком виде(без последнего полносвязного слоя)
# и теперь создаем свои fully connected слои
prediction_layer_1 = layers.Dense(1000, activation='relu')
prediction_layer_2 = layers.Dense(120, activation='softmax')

58892288/58889256 [==============================] - 6s 0us/step


In [26]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 100, 100, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 100, 100, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 50, 50, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 50, 50, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 50, 50, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 25, 25, 128)       0     

In [30]:
# объединяем VGG16 с нашими полносвязными слоями
new_model = tf.keras.Sequential([base_model, layers.Flatten(), layers.Dense(1000, activation='relu'), layers.Dense(120, activation='softmax')]) # new_model=base_model+Flatten()+ pl1+pl2  

In [38]:
# задаем параметры обучения
optimizer = tf.keras.optimizers.Adam(0.001)
new_model.compile(loss='categorical_crossentropy', optimizer = optimizer, metrics=['accuracy'])

In [39]:
# обучаем 
new_model.fit(train_data_gen, steps_per_epoch=features_train_len//16, epochs=5, validation_data=val_data_gen, validation_steps=features_test_len//16)

Epoch 1/5
712/712 [==============================] - 388s 545ms/step - loss: 3.1292 - accuracy: 0.3125 - val_loss: 1.2033 - val_accuracy: 0.6540
Epoch 2/5
712/712 [==============================] - 369s 518ms/step - loss: 0.7664 - accuracy: 0.7824 - val_loss: 0.8626 - val_accuracy: 0.7616
Epoch 3/5
712/712 [==============================] - 363s 509ms/step - loss: 0.4577 - accuracy: 0.8633 - val_loss: 0.4860 - val_accuracy: 0.8521
Epoch 4/5
712/712 [==============================] - 380s 534ms/step - loss: 0.3344 - accuracy: 0.9006 - val_loss: 0.5834 - val_accuracy: 0.8186
Epoch 5/5
712/712 [==============================] - 425s 598ms/step - loss: 0.3151 - accuracy: 0.9015 - val_loss: 0.5615 - val_accuracy: 0.8300


In [40]:
# сохраним модель
new_model.save('/Users/ganievanvar/Tensorflow notebooks/ANNs/CNNs')

INFO:tensorflow:Assets written to: /Users/ganievanvar/Tensorflow notebooks/ANNs/CNNs/assets


In [41]:
new_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 3, 3, 512)         14714688  
_________________________________________________________________
flatten_2 (Flatten)          (None, 4608)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              4609000   
_________________________________________________________________
dense_5 (Dense)              (None, 120)               120120    
Total params: 19,443,808
Trainable params: 4,729,120
Non-trainable params: 14,714,688
_________________________________________________________________
